In [1]:
from flask import Flask, request, jsonify
import cv2
from src.hand_tracker_nms import HandTrackerNMS
import src.extra
import joblib
import numpy as np
import base64

In [1]:
from flask import Flask, request, jsonify
import cv2
from src.hand_tracker_nms import HandTrackerNMS
import src.extra
import joblib
import numpy as np
import base64
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

PALM_MODEL_PATH = "models/palm_detection_without_custom_op.tflite"
LANDMARK_MODEL_PATH = "models/hand_landmark.tflite"
ANCHORS_PATH = "models/anchors.csv"

connections = src.extra.connections
int_to_char = src.extra.classes

detector = HandTrackerNMS(
    PALM_MODEL_PATH,
    LANDMARK_MODEL_PATH,
    ANCHORS_PATH,
    box_shift=0.2,
    box_enlarge=1.3
)

gesture_clf = joblib.load(r'models/gesture_clf.pkl')

word = []
letter = ""
staticGesture = 0

def process_frame(image):
    global letter, staticGesture, word
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    points, bboxes, joints = detector(image)
    if points is not None:
        src.extra.draw_points(points, image)
        pred_sign = src.extra.predict_sign(joints, gesture_clf, int_to_char)
        if letter == pred_sign:
            staticGesture += 1
        else:
            letter = pred_sign
            staticGesture = 0
        if staticGesture > 6:
            word.append(letter)
            staticGesture = 0

    if points is None:
        try:
            if word[-1] != " ":
                staticGesture += 1
                if staticGesture > 6:
                    word.append(" ")
                    staticGesture = 0
        except IndexError:
            print("list 'word' is empty")

    src.extra.draw_sign(word, image, (50, 460))
    return image, word

@app.route('/process', methods=['POST'])
def process():
    data = request.json
    image_data = base64.b64decode(data['image'])
    nparr = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    processed_frame, word = process_frame(image)
    _, buffer = cv2.imencode('.jpg', processed_frame)
    processed_image = base64.b64encode(buffer).decode('utf-8')

    return jsonify({'image': processed_image, 'word': ''.join(word)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.68:5000
Press CTRL+C to quit
